In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [ ]:
file_path = '/content/RIASEC12Q.csv'
try:
  df = pd.read_csv(file_path)
  print("File Loaded!")
except:
  !gdown --id 1Dd6iz13sbZ2Q7IhuwgYS_Y-FzIYoLd7g
  try:
    df = pd.read_csv(file_path)
    print("File Loaded!")
  except:
    print("Something's wrong! check your path!!!")

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Dd6iz13sbZ2Q7IhuwgYS_Y-FzIYoLd7g
To: /content/RIASEC12Q.csv
100% 5.03k/5.03k [00:00<00:00, 14.1MB/s]
File Loaded!


Load Dataset

In [ ]:
df.head(n = 10)

,Unnamed: 0,Jurusan,R1,R2,I1,I2,A1,A2,S1,S2,E1,E2,C1,C2
0,0,Science,0,0,1,1,0,0,2,2,0,0,2,2
1,1,Science,0,0,1,1,0,0,2,2,0,0,2,2
2,2,Arts and Literature,0,0,0,0,1,1,2,2,0,0,0,0
3,3,Economics,0,0,0,0,0,0,0,0,1,1,1,0
4,4,Technology,1,1,0,0,0,0,0,0,0,0,1,1
5,5,Social,0,0,0,0,0,0,1,1,0,0,0,0
6,6,Arts and Literature,0,0,0,0,1,1,2,2,0,0,0,0
7,7,Economics,0,0,0,0,0,0,0,0,1,1,1,0
8,8,Technology,1,1,0,0,0,0,0,0,0,0,1,1
9,9,Social,0,0,0,0,0,0,1,1,0,0,0,0


In [ ]:
# Deleting unused column
try:
  df = df.drop(columns=['Unnamed: 0'])
  print("Columns Dropped!")
except:
  print("Columns Already Dropped!")

Columns Dropped!


In [ ]:
df.head()

,Jurusan,R1,R2,I1,I2,A1,A2,S1,S2,E1,E2,C1,C2
0,Science,0,0,1,1,0,0,2,2,0,0,2,2
1,Science,0,0,1,1,0,0,2,2,0,0,2,2
2,Arts and Literature,0,0,0,0,1,1,2,2,0,0,0,0
3,Economics,0,0,0,0,0,0,0,0,1,1,1,0
4,Technology,1,1,0,0,0,0,0,0,0,0,1,1


Indexing Label

In [ ]:
label = "Jurusan"
classes = [0]

In [ ]:
if type(classes[0]) is int:
  classes = df[label].unique().tolist()
  df[label] = df[label].map(classes.index)
  print(f"Label classes: {classes}")
else:
  print("Label classes Already indexed!")
  print(f"Label classes: {classes}")

Label classes: ['Science', 'Arts and Literature', 'Economics', 'Technology', 'Social']


In [ ]:
df.head()

,Jurusan,R1,R2,I1,I2,A1,A2,S1,S2,E1,E2,C1,C2
0,0,0,0,1,1,0,0,2,2,0,0,2,2
1,0,0,0,1,1,0,0,2,2,0,0,2,2
2,1,0,0,0,0,1,1,2,2,0,0,0,0
3,2,0,0,0,0,0,0,0,0,1,1,1,0
4,3,1,1,0,0,0,0,0,0,0,0,1,1


In [ ]:
# separating labels and features without randomize
X = df.drop("Jurusan", axis=1).values
y = df.Jurusan.values

In [ ]:
# normalization
from sklearn.preprocessing import normalize
X_normalized = normalize(X, axis = 0, norm='l2')

In [ ]:
# splitting data
total_length=len(df)
train_length=int(0.7*total_length)
test_length=int(0.3*total_length)

X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=y[:train_length]
y_test=y[train_length:]

print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

Length of train set x: 91 y: 91
Length of test set x: 39 y: 39


In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=len(classes))
y_test = to_categorical(y_test, num_classes=len(classes))
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Shape of y_train (91, 5)
Shape of y_test (39, 5)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras import regularizers
from keras.optimizers import Adam

shape = X_train.shape[1]

model=Sequential()
model.add(Dense(128,input_dim=X_train.shape[1],activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.6))
# model.add(Dense(32,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(len(classes),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
class cb(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.90 and logs.get('val_loss') < 0.4 and logs.get('loss') < 0.4 and logs.get('val_accuracy') > 0.80):
      self.model.stop_training = True

callbacks = cb()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=350,verbose=1, callbacks=[callbacks])

Epoch 1/350
5/5 [==============================] - 1s 78ms/step - loss: 1.6081 - accuracy: 0.3516 - val_loss: 1.6184 - val_accuracy: 0.3846
Epoch 2/350
5/5 [==============================] - 0s 14ms/step - loss: 1.5561 - accuracy: 0.7253 - val_loss: 1.6190 - val_accuracy: 0.3846
Epoch 3/350
5/5 [==============================] - 0s 15ms/step - loss: 1.5038 - accuracy: 0.8352 - val_loss: 1.6240 - val_accuracy: 0.3846
Epoch 4/350
5/5 [==============================] - 0s 11ms/step - loss: 1.4536 - accuracy: 0.8791 - val_loss: 1.6348 - val_accuracy: 0.3846
Epoch 5/350
5/5 [==============================] - 0s 15ms/step - loss: 1.4124 - accuracy: 0.9011 - val_loss: 1.6446 - val_accuracy: 0.3846
Epoch 6/350
5/5 [==============================] - 0s 10ms/step - loss: 1.3433 - accuracy: 0.8901 - val_loss: 1.6607 - val_accuracy: 0.3846
Epoch 7/350
5/5 [==============================] - 0s 12ms/step - loss: 1.2668 - accuracy: 0.8681 - val_loss: 1.6809 - val_accuracy: 0.3846
Epoch 8/350
5/5 [===

**TESTING**

In [ ]:
science = [0, 0, 1, 1, 0,	0, 2, 2, 0, 0, 2, 2]
arts_and_literature = [0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0]
technology = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

In [ ]:
fitur_normalized = normalize([science], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

1/1 [==============================] - 0s 94ms/step
[0]
top 3 prediksi adalah : [0 3 4]
keseluruhan prediksi adalah : [0 3 4 1 2]
prediksi paling mendekati adalah : [0]
prediksi paling tidak mendekati adalah : 2
label top 3 prediksi adalah : Science, Technology, Social


In [ ]:
fitur_normalized = normalize([arts_and_literature], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

1/1 [==============================] - 0s 25ms/step
[1]
top 3 prediksi adalah : [1 4 0]
keseluruhan prediksi adalah : [1 4 0 3 2]
prediksi paling mendekati adalah : [1]
prediksi paling tidak mendekati adalah : 2
label top 3 prediksi adalah : Arts and Literature, Social, Science


In [ ]:
fitur_normalized = normalize([technology], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

1/1 [==============================] - 0s 22ms/step
[3]
top 3 prediksi adalah : [3 0 4]
keseluruhan prediksi adalah : [3 0 4 2 1]
prediksi paling mendekati adalah : [3]
prediksi paling tidak mendekati adalah : 1
label top 3 prediksi adalah : Technology, Science, Social


In [ ]:
model.save("/content/model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# new_model = tf.keras.models.load_model('/content/model.h5')

In [ ]:
# fitur_normalized = normalize([economics], axis = 0)
# pred = new_model.predict([fitur_normalized])
# np.argmax(pred, axis=1)

In [ ]:
# fitur_normalized = normalize([science], axis = 0)
# pred = new_model.predict([fitur_normalized])
# np.argmax(pred, axis=1)

In [ ]:
# new_model.evaluate(X_test, y_test)

In [ ]:
# # Convert the model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Save the model.
# with open('model_minatku.tflite', 'wb') as f:
#   f.write(tflite_model)